In [1]:
!pip install xgboost

import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
train= pd.read_csv('new_train.csv')
test= pd.read_csv('test.csv')
sub   = pd.read_csv('sample_submission.csv')
train.head()

train.target.value_counts()


0    32542
1      584
Name: target, dtype: int64

# Feature eng (NEW)

In [3]:
train['sex'] = train['sex'].fillna('na')
train['age_approx'] = train['age_approx'].fillna(0)
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('na')

test['sex'] = test['sex'].fillna('na')
test['age_approx'] = test['age_approx'].fillna(0)
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('na')

In [4]:
train['sex'] = train['sex'].astype("category").cat.codes +1
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].astype("category").cat.codes +1
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,tfrecord,width,height,patient_code
0,ISIC_2637011,IP_7279968,2,45.0,1,unknown,benign,0,0,6000,4000,0
1,ISIC_0015719,IP_3075186,1,45.0,7,unknown,benign,0,0,6000,4000,1
2,ISIC_0052212,IP_2842074,1,50.0,2,nevus,benign,0,6,1872,1053,2
3,ISIC_0068279,IP_6890425,1,45.0,1,unknown,benign,0,0,1872,1053,3
4,ISIC_0074268,IP_8723313,1,55.0,7,unknown,benign,0,11,6000,4000,4


In [5]:
test['sex'] = test['sex'].astype("category").cat.codes +1
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].astype("category").cat.codes +1
test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,2,70.0,3
1,ISIC_0052349,IP_7782715,2,40.0,2
2,ISIC_0058510,IP_7960270,1,55.0,6
3,ISIC_0073313,IP_6375035,1,50.0,6
4,ISIC_0073502,IP_0589375,1,45.0,2


In [6]:
x_train = train[['sex', 'age_approx','anatom_site_general_challenge']]
y_train = train['target']


x_test = test[['sex', 'age_approx','anatom_site_general_challenge']]
# y_train = test['target']


train_DMatrix = xgb.DMatrix(x_train, label= y_train)
test_DMatrix = xgb.DMatrix(x_test)

In [12]:
# model = xgb.train(param, 
#                   train_DMatrix, 
#                   num_boost_round=epochs)

clf = xgb.XGBClassifier(n_estimators=2000, 
                        max_depth=8, 
                        objective='multi:softprob',
                        metric='aucpr',
                        seed=8,  
                        nthread=-1, 
                        learning_rate=0.15, 
                        num_class = 2, 
                        scale_pos_weight = (32542/584))



In [13]:
clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.15, max_delta_step=0, max_depth=8, metric='aucpr',
              min_child_weight=1, missing=None, n_estimators=2000, n_jobs=1,
              nthread=-1, num_class=2, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=55.72260273972603, seed=8, silent=None,
              subsample=1, verbosity=1)

In [14]:
a = clf.predict_proba(x_train)[:,1]
a

array([0.01539697, 0.01739808, 0.00974392, ..., 0.01070748, 0.01362674,
       0.00794586], dtype=float32)

In [15]:
roc_auc_score(train['target'].values, a)

0.7462879109652183

In [16]:
# predictions = model.predict(test_DMatrix)
# proba = model.predict_proba(test_DMatrix) 
b = clf.predict_proba(x_test)[:,1]
# clf.predict(x_test)
sub.target = b
sub.head()

,image_name,target
0,ISIC_0052060,0.001979
1,ISIC_0052349,0.008126
2,ISIC_0058510,0.014590
3,ISIC_0073313,0.008271
4,ISIC_0073502,0.007258


In [17]:
sub.to_csv('sub_xgother1.csv', index = False)

https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn